In [1]:
import pandas
import pickle
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import arviz
import pystan
import random
from sklearn.neighbors import KernelDensity
from scipy.spatial import cKDTree as KDTree
from scipy.stats import multivariate_normal, gaussian_kde, entropy
from fastkde import fastKDE
from entropy_estimators import continuous
import seaborn as sns

Helper functions    

In [9]:
def KLdivergence(x, y):
  # Check the dimensions are consistent
  x = np.atleast_2d(x)
  y = np.atleast_2d(y)

  n,d = x.shape
  m,dy = y.shape

  assert(d == dy)

  # Build a KD tree representation of the samples and find the nearest neighbour
  # of each point in x.
  xtree = KDTree(x)
  ytree = KDTree(y)

  # Get the first two nearest neighbours for x, since the closest one is the
  # sample itself.
  r = xtree.query(x, k=2, eps=.01, p=2)[0][:,1] + 1e-100
  s = ytree.query(x, k=1, eps=.01, p=2)[0] + 1e-100

  # There is a mistake in the paper. In Eq. 14, the right side misses a negative sign
  # on the first term of the right hand side.
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))

Build model

In [10]:
# whether to recompile the stan program
DO_COMPILE = False

# stan program path
stan_path = 'multi_feature.stan'

def build_model(path, pkl_file=None, do_compile=True):
    if do_compile:
        sm = pystan.StanModel(file=path)
        if pkl_file is not None:
            with open(pkl_file, 'wb') as f:
                pickle.dump(sm, f)

    # if the program hasn't been complied, check that the file already exists
    else: 
        if os.path.isfile(pkl_file):
            sm = pickle.load(open(pkl_file, 'rb'))
        else:
            raise FileNotFoundError
    return sm


sm = build_model(path = stan_path, pkl_file='model.pkl', do_compile=DO_COMPILE)

Model parameters

In [11]:
# mu
mu_mean = 0
mu_sd = 0.5

# sd
sigma_alpha = 1
sigma_beta = 1

# noise SD prior
noise = 0.25

# environmental EIG
env_info = 0.401

Create data

In [12]:

# number of stimuli
sequence_length = 6

# number of features 
num_features = 1

# number of samples (max)
num_samples = 600

# allocation of samples to exemplars
exemplar_idx = np.repeat(np.arange(1, sequence_length+1), num_samples/sequence_length)

# background / deviant mean values
background = np.repeat(0, num_features)
deviant = np.repeat(3, num_features)

# perceptual noise
sig = np.identity(num_features) * 0.01;

# deviant position
deviant_pos = 2

# stimulus means
exemplar_means = np.tile(background, (sequence_length, 1))
exemplar_means[deviant_pos-1] = deviant

sim_data = [np.random.multivariate_normal(exemplar_means[idx-1], sig) for idx in exemplar_idx] 
sim_data = np.asmatrix(sim_data)

Simulation parameters

In [13]:
# number of iterations and warmup per model run
num_iter = 10000
num_warmup = 3000

# number of total model runs
num_model_runs = 1


Initialize flags, variables, iterators 

In [14]:
# Flags 
sample = True
policy = 'kl' # 'kl', 'surprisal', 'entropy' or 'eig'

# Iterators
samples_from_current_stim = 1
total_samples = 1
exemplar_num = 1

# Variables
model_LT = np.zeros((1, sequence_length))

sample_data = np.empty((num_samples,num_features))
sample_data[:] = np.nan

exemplar_labels = np.empty((num_samples,))
exemplar_labels[:] = np.nan

prior_mu = np.random.multivariate_normal(np.repeat(mu_mean, num_features), np.identity(num_features)*mu_sd, num_iter-num_warmup)

prior_sigma = np.empty((num_iter-num_warmup, num_features))
for i in np.arange(0, num_features):
    prior_sigma[:,i] = np.random.gamma(sigma_alpha, sigma_beta, num_iter-num_warmup)

prior = np.hstack((prior_mu, prior_sigma))

data = {"mu_mean": mu_mean , "mu_sd": mu_sd, "sigma_alpha": sigma_alpha, "sigma_beta": sigma_beta, "noise": noise, "F": num_features}

Action loop

In [15]:
while sample and samples_from_current_stim > 1:
    
    # sample number
    data["M"] = total_samples

    # exemplar number 
    data["K"] = exemplar_num

    # add sim data
    sample_data[total_samples-1] = sim_data[exemplar_idx == exemplar_num][samples_from_current_stim-1]
    data["z"] = np.transpose(sample_data[0:total_samples,:])

    # add exemplar for each id
    exemplar_labels[total_samples-1] = int(exemplar_num)
    data["exemplar_idx"] = [int(x) for x in exemplar_labels[~np.isnan(exemplar_labels)]]

    # get posterior samples
    fit = sm.sampling(data=data, iter=num_iter, chains=1, warmup = num_warmup)

    posterior = np.hstack((fit['mu'], fit['sigma']))
    
    if policy is 'kl':
        # KL divergence between prior and posterior
        stim_info = KLdivergence(posterior, prior)

    elif policy is 'entropy':
        # reduction of entropy
        stim_info = entropy(prior) - entropy(posterior)

    elif policy is 'surprisal':
        # surprisal of current observation given prior
        stim_info = surprisal(prior, sample_data[0:total_samples,:])

    elif policy is 'EIG':
        stim_info = EIG(posterior)
    # generate hypothetical next obs


    # get posterior predictive for each next obs
    # post_preds = 1

    # compute KL for each hypothetical next obs
        # compute_KL(post, pre)


    # get EIG
    # EIG_stim = KLs * post_preds

    # decision rule
    if stim_info < env_info:
        sample = False
        model_LT[exemplar_num-1] = samples_from_current_stim

        # reset/increment counters
        samples_from_current_stim = 1
        exemplar_num =+ 1
    else:
        samples_from_current_stim =+ 1 

    if policy is 'kl' or policy is 'surprisal' or policy is 'entropy':
        prior = posterior
    
    total_samples =+1


model_LT


Gradient evaluation took 1.7e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 10000 [  0%]  (Warmup)
Iteration: 1000 / 10000 [ 10%]  (Warmup)
Iteration: 2000 / 10000 [ 20%]  (Warmup)
Iteration: 3000 / 10000 [ 30%]  (Warmup)
Iteration: 3001 / 10000 [ 30%]  (Sampling)
Iteration: 4000 / 10000 [ 40%]  (Sampling)
Iteration: 5000 / 10000 [ 50%]  (Sampling)
Iteration: 6000 / 10000 [ 60%]  (Sampling)
Iteration: 7000 / 10000 [ 70%]  (Sampling)
Iteration: 8000 / 10000 [ 80%]  (Sampling)
Iteration: 9000 / 10000 [ 90%]  (Sampling)
Iteration: 10000 / 10000 [100%]  (Sampling)

 Elapsed Time: 0.059573 seconds (Warm-up)
               0.169982 seconds (Sampling)
               0.229555 seconds (Total)

